In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', sep='\t', error_bad_lines=False, header=None)

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [3]:
df1 = pd.read_csv('./lastfm-dataset-1K/userid-profile.tsv', sep='\t', error_bad_lines=False, header=None)

In [4]:
len(df)

19098853

In [5]:
df.head()

,0,1,2,3,4,5
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


In [6]:
df.isna().sum()

0          0
1          0
2     600848
3          0
4    2162719
5         12
dtype: int64

In [7]:
df.rename(columns={0: 'user_id', 1: 'timestamp', 2: 'abc', 3: 'artist', 4: 'album', 5: 'song'}, inplace=True)

In [8]:
df.drop(columns=['abc', 'album'], inplace=True)

In [9]:
test = df

In [10]:
test2 = pd.pivot_table(test, values='artist', index=['user_id', 'song', 'artist'], aggfunc=np.count_nonzero)

In [11]:
test3 = pd.DataFrame(test2)

In [12]:
test4 = test3.reset_index()

In [13]:
test4.set_index('user_id', inplace=True)

In [14]:
test4.rename(columns={'timestamp': 'playcount'}, inplace=True)

In [15]:
total_plays_df = test4.groupby('user_id').sum()

In [16]:
total_plays_df.rename(columns={'playcount': 'user_total_plays'}, inplace=True)

In [17]:
song_df = test4.join(total_plays_df)

In [18]:
song_df['score'] = song_df.playcount / song_df.user_total_plays

In [19]:
song_df.head()

,song,artist,playcount,user_total_plays,score
user_id,,,,,
user_000001,'84 Pontiac Dream,Boards Of Canada,14,16685,0.000839
user_000001,(All I Wanted Was To Make You) Feel Good,Bugge Wesseltoft,3,16685,0.000180
user_000001,(D)Ecaying Beauty,Richie Hawtin,1,16685,0.000060
user_000001,(Far From) Home,Tiga,1,16685,0.000060
user_000001,(Get Off Your) High Horse Lady,Oasis,1,16685,0.000060


In [20]:
percentile = pd.DataFrame(song_df.groupby('user_id').score.quantile(.99))

In [21]:
test = song_df.join(percentile, rsuffix='99percentile')


In [22]:
song_df_above_99 = test[test['score'] >= test['score99percentile']]
song_df_above_99.head()

,song,artist,playcount,user_total_plays,score,score99percentile
user_id,,,,,,
user_000001,All That You Give,The Cinematic Orchestra,38,16685,0.002277,0.001738
user_000001,Beautiful Burnout,Underworld,33,16685,0.001978,0.001738
user_000001,Behind The Mask (Live_2009_4_15),坂本龍一,34,16685,0.002038,0.001738
user_000001,"Boy, Boy, Boy",Underworld,32,16685,0.001918,0.001738
user_000001,Breezin',Cornelius,30,16685,0.001798,0.001738


In [23]:
song_df_above_99.drop(columns=['score99percentile'], inplace=True)
song_df_above_99.head()

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,song,artist,playcount,user_total_plays,score
user_id,,,,,
user_000001,All That You Give,The Cinematic Orchestra,38,16685,0.002277
user_000001,Beautiful Burnout,Underworld,33,16685,0.001978
user_000001,Behind The Mask (Live_2009_4_15),坂本龍一,34,16685,0.002038
user_000001,"Boy, Boy, Boy",Underworld,32,16685,0.001918
user_000001,Breezin',Cornelius,30,16685,0.001798


In [24]:
len(song_df_above_99)

55715

In [25]:
song_artist_list = list(zip(song_df_above_99.song, song_df_above_99.artist))

In [26]:
song_artist_list = list(set(song_artist_list))

In [27]:
len(song_artist_list)

42193

### Spotify API

In [28]:
import requests
client_id = '8787e23c39884c2b98026feff5b7acde'
client_secret = '94d8c07f39f841739a58e5e12c9e5e86'

grant_type = 'client_credentials'

#Request based on Client Credentials Flow from https://developer.spotify.com/web-api/authorization-guide/

#Request body parameter: grant_type Value: Required. Set it to client_credentials
body_params = {'grant_type' : grant_type}

url='https://accounts.spotify.com/api/token'

response=requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.text)

{"access_token":"BQDYp_ROOiGd3GExGvD0peozlL9L0BTnPf7dvLUcH10rAcOOqrEGRfqdxdIWy3G31oPdMNfHed1Lw0D4vDM","token_type":"Bearer","expires_in":3600,"scope":""}


In [29]:
access_token = 'BQAJzQnMftpBsuemHdkzH3ws85n46UTn8KJCLBKjtOsOeyw-OgyC2foRqKC95nOFLfBUJgebDlB4DkqUDGc'
headers = {'Authorization': 'Bearer '+ access_token}

In [30]:
from tqdm import tqdm
def get_track_ids(tracklist):
    id_list = {}
    for track in tqdm(tracklist):
        artist = track[1].replace(" ", "%20")
        song = track[0].replace(" ", "%20")
        try:
            r = requests.get(f'https://api.spotify.com/v1/search?q=artist:{artist}%20track:{song}&type=track', headers=headers)
            data = r.json()
            track_id = data['tracks']['items'][0]['id']
            id_list[track] = track_id
        except:
            id_list[track] = "NA"
    return id_list

* spotify access tokens expire every hour so run the following  cells in increments, refreshing token after each set

In [31]:
# first_set_ids = get_track_ids(song_artist_list[:10000])

In [32]:
# second_set_ids = get_track_ids(song_artist_list[10000:20000])

In [33]:
# third_set_ids = get_track_ids(song_artist_list[20000:30000])

In [34]:
# fourth_set_ids = get_track_ids(song_artist_list[30000:])

In [35]:
# first_set_ids = [(x,y) for x,y in first_set_ids.items() if y != 'NA']
# second_set_ids = [(x,y) for x,y in second_set_ids.items() if y != 'NA']
# third_set_ids = [(x,y) for x,y in third_set_ids.items() if y != 'NA']
# fourth_set_ids = [(x,y) for x,y in fourth_set_ids.items() if y != 'NA']

#### getting audio features from spotify API

In [36]:
def get_track_features(track_id):
    r = requests.get(f'https://api.spotify.com/v1/audio-features/{track_id}', headers=headers)
    data = r.json()
    return data

In [37]:
# features = {}
# for x, y in tqdm(first_set_ids):
#     try:
#         features[x] = get_track_features(y)
#     except:
#         features[x] = 'NA'

In [38]:
# for x, y in tqdm(second_set_ids):
#     try:
#         features[x] = get_track_features(y)
#     except:
#         features[x] = 'NA'

In [39]:
# for x, y in tqdm(third_set_ids):
#     try:
#         features[x] = get_track_features(y)
#     except:
#         features[x] = 'NA'

In [40]:
# for x, y in tqdm(fourth_set_ids):
#     try:
#         features[x] = get_track_features(y)
#     except:
#         features[x] = 'NA'

In [41]:
# features_df = pd.DataFrame(features).T

In [42]:
# features_df.reset_index(inplace=True)

In [43]:
# ids = first_set_ids+second_set_ids+third_set_ids+fourth_set_ids

In [44]:
# features_df.index = [x[0] for x in ids]
# features_df['spotify_id'] = [x[1] for x in ids]

In [45]:
# features_df.columns

In [46]:
# features_df.drop(columns=['level_0', 'level_1', 'analysis_url', 'error', 'track_href', 'type', 'id', 'uri'], inplace=True)

In [47]:
# features_df.to_csv('features_df.csv')

In [80]:
# features_df = pd.read_csv('features_df.csv')

In [59]:
# features_df.set_index('Unnamed: 0', inplace=True)

In [102]:
# features_df['Unnamed: 0'] = [a.split(",") for a in features_df['Unnamed: 0']]

In [103]:
# features_df['Unnamed: 0'] = [(a[0].replace("('", "").replace("'",""), a[1].replace(" '", "").replace("')","")) for a in features_df['Unnamed: 0']]

In [105]:
# features_df.set_index('Unnamed: 0', inplace=True)

In [66]:
# songs = [x for x in song_df_above_99.song]
# artist = [x for x in song_df_above_99.artist]
# song_artist = [(x,y) for x,y in zip(songs, artist)]

In [68]:
# song_df_above_99['song_artist'] = song_artist

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [106]:
# final_table = song_df_above_99.join(features_df, on='song_artist')

In [109]:
# final_table.dropna(inplace=True)

In [110]:
# len(final_table)

40846

In [113]:
# final_table.to_csv('final_table.csv',quoting=None)

In [115]:
final_table = pd.read_csv('final_table.csv')

In [116]:
final_table

,user_id,song,artist,playcount,user_total_plays,score,song_artist,acousticness,danceability,duration_ms,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,spotify_id
0,user_000001,All That You Give,The Cinematic Orchestra,38,16685,0.002277,"('All That You Give', 'The Cinematic Orchestra')",0.157000,0.575,370573.0,...,0.006110,9.0,0.2410,-9.949,1.0,0.0277,125.411,4.0,0.1330,6vifb1Bh7078BRcVixLRyQ
1,user_000001,Beautiful Burnout,Underworld,33,16685,0.001978,"('Beautiful Burnout', 'Underworld')",0.093700,0.648,489267.0,...,0.804000,4.0,0.0964,-11.793,0.0,0.0439,130.002,4.0,0.3300,1I61h0TNyzE86dx1tZzj3b
2,user_000001,Child Song,The Cinematic Orchestra,45,16685,0.002697,"('Child Song', 'The Cinematic Orchestra')",0.672000,0.545,314453.0,...,0.799000,4.0,0.1140,-10.202,0.0,0.0304,89.867,1.0,0.2270,1KcZeJiC4AXDLZMTjfeaid
3,user_000001,Clap & Whistle & Walking,Cornelius,33,16685,0.001978,"('Clap & Whistle & Walking', 'Cornelius')",0.000377,0.794,149304.0,...,0.734000,5.0,0.0974,-7.125,1.0,0.0600,105.968,4.0,0.1200,40JDCgTZHWPozHzovLQXjy
4,user_000001,Everything Is Alright,Rasmus Faber,31,16685,0.001858,"('Everything Is Alright', 'Rasmus Faber')",0.036400,0.688,216413.0,...,0.217000,8.0,0.1290,-7.506,1.0,0.0286,120.015,4.0,0.2270,25bAi7EIMJWsA6dQccY8cA
5,user_000001,Familiar Ground,The Cinematic Orchestra,31,16685,0.001858,"('Familiar Ground', 'The Cinematic Orchestra')",0.324000,0.472,276093.0,...,0.139000,11.0,0.1330,-13.836,0.0,0.0365,125.966,4.0,0.0802,4KbNjU82q7fX494OGIPg3Z
6,user_000001,Flite,The Cinematic Orchestra,41,16685,0.002457,"('Flite', 'The Cinematic Orchestra')",0.165000,0.541,395707.0,...,0.921000,11.0,0.1080,-11.082,0.0,0.0363,137.033,1.0,0.3210,1g7bl2LDCom7jtX2EmHRSF
7,user_000001,Glam Bucket,Underworld,31,16685,0.001858,"('Glam Bucket', 'Underworld')",0.181000,0.356,345933.0,...,0.388000,9.0,0.0958,-13.236,0.0,0.0806,125.983,4.0,0.1200,7k5rnk06tw2kUOj7AQXkha
8,user_000001,Gum,Cornelius,63,16685,0.003776,"('Gum', 'Cornelius')",0.000014,0.512,225760.0,...,0.814000,1.0,0.1550,-5.147,0.0,0.0694,94.988,4.0,0.1680,6DwiGAiOMvLaYHjd6nhpL1
9,user_000001,I Can See,Jazzanova Feat. Ben Westbeech,29,16685,0.001738,"('I Can See', 'Jazzanova Feat. Ben Westbeech')",0.010600,0.706,505773.0,...,0.000787,7.0,0.1760,-7.207,1.0,0.0558,119.980,4.0,0.7300,56ZE1psL7yFIg0j2pAkSSJ
